In [1]:
# to make the export go to the fitbit app, click on your icon, and order a google takeout 
# took like 8 hours the last time

In [2]:
import os
import sys
def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")+1]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath)
from utils import exportsDataPath, workingDataPath, writeHypnoDfParquet

import pandas as pd

fitbitHRWorkingDataPath = workingDataPath + 'fitbit/hr/'

# Get the list of all files and directories
exportFilePath = exportsDataPath + "27-8-24/takeout-20240828T033834Z-001/Takeout/Fitbit/Global Export Data/"
dir_list = os.listdir(exportFilePath)

NameError: name 'pathOfExport' is not defined

In [3]:
# read in the existing files and make the existing df
workingDataFiles = os.listdir(fitbitHRWorkingDataPath)
columnNames = ["sampleDT", "confidence", "value"]
dfSoFar = pd.DataFrame(columns=columnNames).set_index("sampleDT")

for dataFileName in workingDataFiles:
    dfSoFar = pd.concat([dfSoFar, pd.read_parquet(fitbitHRWorkingDataPath + dataFileName)]) 

dfSoFar = dfSoFar[~dfSoFar.index.duplicated(keep="first")].sort_index()

print(f"read in {len(dfSoFar)} rows from {len(workingDataFiles)} files")


read in 0 rows from 0 files


In [6]:
# get a list of unique dates in the index
#   removing the latest 3 days since they might be incomplete
if len(dfSoFar) > 0:
    datesSoFar = sorted(list(set(dfSoFar.index.date)))[:-3]
    print(datesSoFar[-1])
    hrFilenames = [x for x in dir_list if x.split("_")[0] == "heart"]
    hrFilenames = [x for x in hrFilenames if pd.to_datetime(x[11:-5]).date() not in datesSoFar]
    hrFilenames

In [7]:
#takes like 40 minutes to make 15.8 million rows
import json

columnNames = ["sampleDT", "confidence", "value"]

samplesCount = 0
samplesList = []

for fn in hrFilenames:
    data = json.load(open(exportFilePath + fn))
    for row in data:
        samplesList.append([pd.to_datetime(row["dateTime"] + "+0000").tz_convert("US/Arizona"), row["value"]["confidence"], row["value"]["bpm"]])
        samplesCount += 1
        if samplesCount % 100_000 == 0:
            print(f"added {samplesCount} samples so far")
print(f"added {samplesCount} samples")

added 100000 samples so far
added 200000 samples so far
added 300000 samples so far
added 400000 samples so far
added 500000 samples so far
added 600000 samples so far
added 700000 samples so far
added 800000 samples so far
added 900000 samples so far
added 1000000 samples so far
added 1100000 samples so far
added 1200000 samples so far
added 1300000 samples so far
added 1400000 samples so far
added 1500000 samples so far
added 1600000 samples so far
added 1700000 samples so far
added 1800000 samples so far
added 1900000 samples so far
added 2000000 samples so far
added 2100000 samples so far
added 2200000 samples so far
added 2300000 samples so far
added 2400000 samples so far
added 2500000 samples so far
added 2600000 samples so far
added 2700000 samples so far
added 2800000 samples so far
added 2900000 samples so far
added 3000000 samples so far
added 3100000 samples so far
added 3200000 samples so far
added 3300000 samples so far
added 3400000 samples so far
added 3500000 samples s

In [8]:
samplesList = sorted(samplesList, key=lambda x: x[0]) #sort by timestamp

In [10]:
import pandas as pd

fitbitHRdf = pd.DataFrame(data=samplesList, columns=columnNames)
fitbitHRdf = fitbitHRdf.set_index("sampleDT")
fitbitHRdf["confidence"] = fitbitHRdf["confidence"].astype('uint8')
fitbitHRdf["value"] = fitbitHRdf["value"].astype('uint8')
fitbitHRdf.dtypes

confidence    uint8
value         uint8
dtype: object

In [11]:
# concatenate the df's
# remove duplicate indexes
fitbitHRdf = pd.concat([dfSoFar, fitbitHRdf])
fitbitHRdf = fitbitHRdf[~fitbitHRdf.index.duplicated(keep="first")].sort_index()

In [ ]:
writeWorkingHRDfParquet('fitbit', fitbitHRdf)

In [12]:
# Get the size of the file in bytes
workingDataCompleteFile = "/home/chowder/Documents/workingData/fitbit/hr/fitbitHRdf.parquet.gzip"
fitbitHRdf.to_parquet(workingDataCompleteFile,
              compression='gzip') 

file_size = os.path.getsize(workingDataCompleteFile)
os.remove(workingDataCompleteFile)

In [ ]:
import math
numFiles = math.ceil(file_size / (1024 * 1024 * 5))
rows_per_file = int(len(fitbitHRdf)/numFiles)

print(f"the file size of all the data is about {file_size // (1024 * 1024)} MB")
print(f"the total number of rows in the file is {len(fitbitHRdf)}")
print(f"splitting into {numFiles} number of files with {rows_per_file} rows per file")

for fileNumber in range(numFiles + 1):
    startRow = fileNumber * rows_per_file
    if fileNumber == numFiles:
        endRow = len(fitbitHRdf) - 1
    else:
        endRow = ((fileNumber + 1) * rows_per_file) - 1

    print(f"saving rows {startRow} to {endRow}")
    print(fitbitHRdf.iloc[startRow])

    parquetName = fitbitHRdf.iloc[startRow].name.strftime('%Y-%m-%dT%H%M%S%z') +\
                  "_" +\
                  fitbitHRdf.iloc[endRow].name.strftime('%Y-%m-%dT%H%M%S%z') +\
                  ".parquet.gzip"
    print(f"to a file named {parquetName}")

    print(pd.to_datetime(fitbitHRdf.iloc[startRow].name.strftime('%Y-%m-%dT%H%M%S%z')))

    fitbitHRdf.iloc[startRow:endRow+1].to_parquet(workingDataPath + parquetName,
              compression='gzip') 


the file size of all the data is about 86 MB
the total number of rows in the file is 15826297
splitting into 18 number of files with 879238 rows per file

saving rows 0 to 879237
confidence     1
value         70
Name: 2020-05-13 17:02:34-07:00, dtype: object
to a file named 2020-05-13T170234-0700_2020-08-02T072756-0700.parquet.gzip
2020-05-13 17:02:34-07:00
saving rows 879238 to 1758475
confidence     3
value         67
Name: 2020-08-02 07:28:51-07:00, dtype: object
to a file named 2020-08-02T072851-0700_2020-10-22T191111-0700.parquet.gzip
2020-08-02 07:28:51-07:00
saving rows 1758476 to 2637713
confidence     2
value         87
Name: 2020-10-22 19:12:06-07:00, dtype: object
to a file named 2020-10-22T191206-0700_2021-01-11T152818-0700.parquet.gzip
2020-10-22 19:12:06-07:00
saving rows 2637714 to 3516951
confidence     3
value         60
Name: 2021-01-11 15:28:33-07:00, dtype: object
to a file named 2021-01-11T152833-0700_2021-03-31T170238-0700.parquet.gzip
2021-01-11 15:28:33-07:00
s